# Test Code Functionality
> Functions to test code snippets using the library:

In [ ]:
#| default_exp cli.test_code

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import tempfile
import subprocess
import sys
import os
from pathlib import Path
from typing import Dict, List, Tuple, Any, Optional

from cjm_fasthtml_tailwind.cli.utils import discover_utility_modules
from cjm_fasthtml_tailwind.cli.factory_extraction import extract_factories_from_module
from cjm_fasthtml_tailwind.cli.helper_discovery import get_module_helpers

In [ ]:
#| export
def create_test_script(code: str) -> str:
    """Create a test script with necessary imports and the provided code."""
    # Build the import statements dynamically based on discovered modules
    imports = ["from fasthtml.common import *"]
    
    # Add imports for factories from each utility module
    for module_name, module in discover_utility_modules():
        # Get the factory names from this module
        factories = extract_factories_from_module(module, module_name)
        if factories:
            # Import only the actual factory names
            factory_names = [f.name for f in factories]
            import_list = ", ".join(factory_names)
            imports.append(f"from cjm_fasthtml_tailwind.utilities.{module_name} import {import_list}")
        
        # Also check for helper functions and import them
        helpers = get_module_helpers(module_name)
        if helpers:
            helper_names = [h.name for h in helpers]
            # Check if we already have factories to import
            if factories:
                # Combine with existing import
                all_names = factory_names + helper_names
                import_list = ", ".join(all_names)
                # Replace the last import for this module
                imports[-1] = f"from cjm_fasthtml_tailwind.utilities.{module_name} import {import_list}"
            else:
                # Just import helpers
                import_list = ", ".join(helper_names)
                imports.append(f"from cjm_fasthtml_tailwind.utilities.{module_name} import {import_list}")
    
    # Add any other common imports
    imports.extend([
        "from cjm_fasthtml_tailwind.core.base import combine_classes",
        "from cjm_fasthtml_tailwind.core.resources import get_tailwind_headers",
    ])
    
    # Create the full script
    script = "\n".join(imports) + "\n\n"
    script += "# User code starts here\n"
    script += code + "\n"
    
    return script

In [ ]:
#| export
def execute_test_code(
    code: str,  # The code to test
    show_imports: bool = False,  # Whether to show the generated imports
    timeout: int = 10  # Timeout in seconds
) -> Tuple[bool, str, str]:  # (success, stdout, stderr)
    """Execute test code in a safe environment."""
    # Create the test script
    script = create_test_script(code)
    
    if show_imports:
        print("Generated script:")
        print("=" * 60)
        print(script)
        print("=" * 60)
        print()
    
    # Create a temporary file
    with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
        f.write(script)
        temp_file = f.name
    
    try:
        # Execute the script
        result = subprocess.run(
            [sys.executable, temp_file],
            capture_output=True,
            text=True,
            timeout=timeout
        )
        
        success = result.returncode == 0
        return success, result.stdout, result.stderr
        
    except subprocess.TimeoutExpired:
        return False, "", f"Code execution timed out after {timeout} seconds"
    except Exception as e:
        return False, "", f"Error executing code: {str(e)}"
    finally:
        # Clean up the temporary file
        try:
            os.unlink(temp_file)
        except:
            pass

## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()